<h1 align="center"><font color="yellow">Raspando (Scraping) a Web para aplicativos de PNL</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

Este script esta baseado no site [The AiEdge NEWSLETTER do Damien Benveniste](https://newsletter.theaiedge.io/)

# Contexto

Hoje em dia, é fácil encontrar dados em qualquer lugar! Vamos vasculhar a web para extrair informações sobre os diferentes ganhadores do `Prêmio Nobel de Física`. Podemos encontrar essa lista na [primeira tabela deste link na Wikipédia](https://en.wikipedia.org/wiki/List_of_Nobel_laureates_in_Physics).

![](https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fa9769d35-7e7a-4d89-895f-f5cf3a814811_500x492.png)

Vamos extrair e usar os dados de texto de cada página da Wikipedia relacionada a esses laureados para construir `métricas de similaridade` e tentar prever qual `Física` eles praticaram.

![](https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F678e6fcf-94ea-4963-8f04-b600463db0c9_3178x2126.png)

# Preparando o Dataset dos Ganhadores do Prêmio Nobel

## <font color="red">Obtendo os Dados</font>

Vamos obter os dados de todos os laureados com o [Prêmio Nobel de Física da Wikipedia](https://en.wikipedia.org/wiki/List_of_Nobel_laureates_in_Physics). Podemos usar [pd.read_html](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html) para ler diretamente as tabelas da Wikipédia, assim:

In [62]:
import pandas as pd

url = "https://en.wikipedia.org/wiki/List_of_Nobel_laureates_in_Physics"

# Fornecemos a URL ao Pandas:
nobel_tables = pd.read_html(url)

# Pegamos a primeira Tabela --> 0
nobel_df = nobel_tables[0]
nobel_df.head()

,Year,Image,Laureate[A],Country[B],Rationale[C],Ref
0,1901,NaN,Wilhelm Röntgen (1845–1923),German Empire,"""in recognition of the extraordinary services ...",[19]
1,1902,NaN,Hendrik Lorentz (1853–1928),Netherlands,"""in recognition of the extraordinary service t...",[20]
2,1902,NaN,Pieter Zeeman (1865–1943),Netherlands,"""in recognition of the extraordinary service t...",[20]
3,1903,NaN,Antoine Henri Becquerel (1852–1908),France,"""for his discovery of spontaneous radioactivity""",[21]
4,1903,NaN,Pierre Curie (1859–1906),France,"""for their joint researches on the radiation p...",[21]


In [63]:
nobel_df.shape

(228, 6)

## <font color="red">Limpeza de Dados</font>

<font color="orange">Começamos eliminando as colunas que não são relevantes para a nossa análise:</font>

In [64]:
# Eliminamos duas colunas:
nobel_df.drop(['Image', 'Ref'], axis=1, inplace=True)

<font color="orange">Limpamos os nomes das colunas e os traduzimos ao Português:</font>

In [65]:
nobel_df.columns = ['Ano', 'Laureado', 'País', 'Justificativa']

<font color="orange">Eliminamos todas as linhas em que o prêmio Nobel não foi concedido:</font>

In [66]:
nobel_df = nobel_df.loc[~nobel_df['Laureado'].str.startswith('Not awarded')]

# Sempre tem que aplicar o RESET para os Índices, para que eles sejam novamente en forma crescente:
nobel_df.reset_index(drop=True, inplace=True)


In [67]:
nobel_df.head()

,Ano,Laureado,País,Justificativa
0,1901,Wilhelm Röntgen (1845–1923),German Empire,"""in recognition of the extraordinary services ..."
1,1902,Hendrik Lorentz (1853–1928),Netherlands,"""in recognition of the extraordinary service t..."
2,1902,Pieter Zeeman (1865–1943),Netherlands,"""in recognition of the extraordinary service t..."
3,1903,Antoine Henri Becquerel (1852–1908),France,"""for his discovery of spontaneous radioactivity"""
4,1903,Pierre Curie (1859–1906),France,"""for their joint researches on the radiation p..."


In [68]:
nobel_df.shape

(222, 4)

<font color="orange">Verificamos que não temos Dados Missing: </font>

In [76]:
nobel_df.isnull().any()

Ano              False
Laureado         False
País             False
Justificativa    False
dtype: bool

In [77]:
# Também podemos usar
nobel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222 entries, 0 to 221
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Ano            222 non-null    int64 
 1   Laureado       222 non-null    object
 2   País           222 non-null    object
 3   Justificativa  222 non-null    object
dtypes: int64(1), object(3)
memory usage: 7.1+ KB


# <font color="red">Explorando nossos Dados com algumas perguntas</font>

## <font color="pink">Quantos físicos ganharam um prêmio Nobel?</font>

<font color="orange">Primeiro vou remover as datas da coluna "Laureado", para isso começamos removendo qualquer coisa entre os parênteses usando [RegEx](https://medium.com/xp-inc/regex-um-guia-pratico-para-express%C3%B5es-regulares-1ac5fa4dd39f).</font>

In [78]:
# Assim:
nobel_df['Laureado'] = nobel_df['Laureado'].str.replace('\(.+?\)', '', regex=True)



# ou assim:

# import re

# def Delete_Date(text):
#     text = re.sub(r'\(.+?\)', '', text)
#     return text


# nobel_df['Laureado'] = nobel_df['Laureado'].apply(lambda x: Delete_Date(x))

/tmp/ipykernel_22144/1741914262.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nobel_df['Laureado'] = nobel_df['Laureado'].str.replace('\(.+?\)', '', regex=True)


In [79]:
nobel_df.head()

,Ano,Laureado,País,Justificativa
0,1901,Wilhelm Röntgen,German Empire,"""in recognition of the extraordinary services ..."
1,1902,Hendrik Lorentz,Netherlands,"""in recognition of the extraordinary service t..."
2,1902,Pieter Zeeman,Netherlands,"""in recognition of the extraordinary service t..."
3,1903,Antoine Henri Becquerel,France,"""for his discovery of spontaneous radioactivity"""
4,1903,Pierre Curie,France,"""for their joint researches on the radiation p..."


In [80]:
# Removemos os espaços em branco desnecessários:
nobel_df['Laureado'] = nobel_df['Laureado'].str.strip()


/tmp/ipykernel_22144/1780877785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nobel_df['Laureado'] = nobel_df['Laureado'].str.strip()


In [81]:
nobel_df.head()

,Ano,Laureado,País,Justificativa
0,1901,Wilhelm Röntgen,German Empire,"""in recognition of the extraordinary services ..."
1,1902,Hendrik Lorentz,Netherlands,"""in recognition of the extraordinary service t..."
2,1902,Pieter Zeeman,Netherlands,"""in recognition of the extraordinary service t..."
3,1903,Antoine Henri Becquerel,France,"""for his discovery of spontaneous radioactivity"""
4,1903,Pierre Curie,France,"""for their joint researches on the radiation p..."
